In [1]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.interpolate import interp1d
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, confusion_matrix

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# 랜드마크 인덱스 정의
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# 데이터 증강 함수 정의
def augment_sequence(sequence, factor=0.2):
    time_warped = []
    for landmark in sequence:
        x = np.arange(len(landmark))
        f = interp1d(x, landmark, kind='linear', axis=0)
        x_new = np.linspace(0, len(landmark) - 1, num=int(len(landmark) * (1 + factor)))
        time_warped.append(f(x_new))
    return np.array(time_warped)

# 데이터셋 클래스 정의
class FallSequenceDataset(Dataset):
    def __init__(self, json_files, sequence_length=3):
        self.sequence_length = sequence_length
        self.sequences = []
        self.labels = []
        self.scaler = StandardScaler()
        print(f"LANDMARKS length: {len(LANDMARKS)}")
    
        all_landmarks = []
        
        for json_file in json_files:
            print(f'Processing file: {json_file}')
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            frames = list(data['pose_data'].values())
            
            for i in range(0, len(frames) - self.sequence_length + 1):
                sequence = frames[i:i+self.sequence_length]
                landmarks = []
                fall_frames = 0
                
                for frame in sequence:
                    frame_landmarks = []
                    for landmark in LANDMARKS:
                        frame_landmarks.extend([
                            frame[f'landmark_{landmark}']['x'],
                            frame[f'landmark_{landmark}']['y']
                        ])
                    landmarks.append(frame_landmarks)
                    if frame['class'] == 'Fall':
                        fall_frames += 1
                # 데이터 증강 적용        
                augmented_sequence = augment_sequence(landmarks)            
                all_landmarks.extend(augmented_sequence)
                
                # 레이블 재정의
                if fall_frames == 0:
                    label = 0  # 비낙상
                elif fall_frames == self.sequence_length:
                    label = 2  # 완전 낙상
                else:
                    label = 1  # 낙상 위험
                
                self.sequences.append(augmented_sequence)
                self.labels.append(label)
        
        # 전체 데이터 정규화
        all_landmarks = np.array(all_landmarks)
        all_landmarks_scaled = self.scaler.fit_transform(all_landmarks)
        
        # 정규화된 데이터를 다시 시퀀스로 재구성
        for i in range(len(self.sequences)):
            start = i * self.sequence_length
            end = start + self.sequence_length
            self.sequences[i] = all_landmarks_scaled[start:end]
        
        if self.sequences:
            print(f"Sample sequence shape: {self.sequences[0].shape}")
        else:
            print("No sequences were created.")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        if idx >= len(self.sequences):
            raise IndexError(f"Index {idx} out of range. Dataset length: {len(self.sequences)}")
        sequence = self.sequences[idx]
        return torch.FloatTensor(sequence), torch.LongTensor([self.labels[idx]]).squeeze()

# GRU 모델 정의
class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=3, dropout = 0.5):
        super(FallDetectionGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout = dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[ : , -1, :])
        out = self.fc(out)
        return out

# 데이터 로드 및 전처리
json_folder = 'D:\\human_fall\\re_landmark\\re_train_NY_json'
json_files = [os.path.join(json_folder, f) for f in os.listdir(json_folder) if f.endswith('.json')]
dataset = FallSequenceDataset(json_files)

# 데이터셋을 학습 및 검증 세트로 분할
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

# 데이터 로더 생성 전에 클래스 가중치 계산
class_weights = compute_class_weight('balanced', classes=np.unique(dataset.labels), y=dataset.labels)
class_weights = torch.FloatTensor(class_weights).to(device)

# 손실 함수에 가중치 적용
criterion = nn.CrossEntropyLoss(weight=class_weights)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

if len(dataset) > 0:
    sample_sequence, sample_label = dataset[0]
    input_size = sample_sequence.shape[1]
    model = FallDetectionGRU(input_size).to(device)
else:
    print("데이터 없음")
    exit()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,  weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
num_epochs = 500
best_loss = float('inf')
patience = 50 # 15 이후 다시 실행을 위해 50으로 변경
no_improve = 0

for epoch in range(num_epochs):
    model.train()
    total_loss_train = 0
    
    for sequences, labels in train_loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(sequences)
        loss_train = criterion(outputs, labels.view(-1))
        loss_train.backward()
        optimizer.step()
        
        total_loss_train += loss_train.item()
    
    avg_loss_train = total_loss_train / len(train_loader)

    # 검증 단계 추가
    model.eval()
    total_loss_val = 0
    
    with torch.no_grad():
        for sequences_val, labels_val in val_loader:
            sequences_val, labels_val = sequences_val.to(device), labels_val.to(device)
            outputs_val = model(sequences_val)
            loss_val = criterion(outputs_val, labels_val.view(-1))
            total_loss_val += loss_val.item()
    
    avg_loss_val = total_loss_val / len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss_train:.4f}, Val Loss: {avg_loss_val:.4f}')
    scheduler.step(avg_loss_val)
    
    if avg_loss_val < best_loss:
        best_loss = avg_loss_val
        no_improve = 0
        torch.save(model.state_dict(), 'try_2.pt')
    else:
        no_improve += 1
    
    if no_improve >= patience:
        print("Early stopping")
        break
    
def calculate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for sequences, labels in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)
    return f1, cm

# 학습 루프 내에서 성능 지표 계산
train_f1, train_cm = calculate_metrics(model, train_loader)
val_f1, val_cm = calculate_metrics(model, val_loader)
print(f'Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')
print(f'Train CM:\n{train_cm}\nVal CM:\n{val_cm}')

print("Training completed")

LANDMARKS length: 11
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C1.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C2.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C3.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C4.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C5.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C6.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C7.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00060_H_A_SY_C8.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00093_H_A_FY_C1.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00093_H_A_FY_C2.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00093_H_A_FY_C3.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00093_H_A_FY_C4.json
Process

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/500], Train Loss: 0.7129, Val Loss: 0.6668
Epoch [2/500], Train Loss: 0.6764, Val Loss: 0.6457
Epoch [3/500], Train Loss: 0.6613, Val Loss: 0.6350
Epoch [4/500], Train Loss: 0.6481, Val Loss: 0.6253
Epoch [5/500], Train Loss: 0.6374, Val Loss: 0.6105
Epoch [6/500], Train Loss: 0.6251, Val Loss: 0.5941
Epoch [7/500], Train Loss: 0.6151, Val Loss: 0.5875
Epoch [8/500], Train Loss: 0.6053, Val Loss: 0.5802
Epoch [9/500], Train Loss: 0.5964, Val Loss: 0.5714
Epoch [10/500], Train Loss: 0.5894, Val Loss: 0.5622
Epoch [11/500], Train Loss: 0.5820, Val Loss: 0.5607
Epoch [12/500], Train Loss: 0.5745, Val Loss: 0.5503
Epoch [13/500], Train Loss: 0.5659, Val Loss: 0.5476
Epoch [14/500], Train Loss: 0.5599, Val Loss: 0.5390
Epoch [15/500], Train Loss: 0.5537, Val Loss: 0.5355
Epoch [16/500], Train Loss: 0.5476, Val Loss: 0.5274
Epoch [17/500], Train Loss: 0.5417, Val Loss: 0.5211
Epoch [18/500], Train Loss: 0.5377, Val Loss: 0.5197
Epoch [19/500], Train Loss: 0.5338, Val Loss: 0.5193
Ep

### sensordata를 포함해 클래스를 재정의한 json 파일로 다시 훈련

In [ ]:
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# 데이터 증강 삭제
# 데이터셋 클래스 정의
class FallSequenceDataset(Dataset):
    def __init__(self, json_files, sequence_length=3):
        self.sequence_length = sequence_length
        self.sequences = []
        self.labels = []
        self.scaler = StandardScaler()
        print(f"LANDMARKS length: {len(LANDMARKS)}")
        
        if self.sequences:
            print(f"sequence shape : {self.sequences[0].shape}")
            print(f"랜드마크 수 : {len(LANDMARKS)}")
            print(f"Features / landmark: {self.sequences[0].shape[1] // len(LANDMARKS)}")
            print(f"전체 features per frame: {self.sequences[0].shape[1]}")
        else:
            print("시퀀스 생성 실패")
    
        all_landmarks = []
        
        for json_file in json_files:
            print(f'Processing file: {json_file}')
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            frames = list(data['pose_data'].values())
            
            for i in range(0, len(frames) - self.sequence_length + 1):
                sequence = frames[i:i+self.sequence_length]
                landmarks = []
                
                for frame in sequence:
                    frame_landmarks = []
                    for landmark in LANDMARKS:
                        frame_landmarks.extend([
                            frame[f'landmark_{landmark}']['x'],
                            frame[f'landmark_{landmark}']['y']
                        ])
                    landmarks.append(frame_landmarks)
                
                all_landmarks.extend(landmarks)
                
                # 마지막 프레임의 클래스를 레이블로 사용
                label = 0 if frame['class'] == 'Normal' else (1 if frame['class'] == 'Danger' else 2)
                
                self.sequences.append(landmarks)
                self.labels.append(label)
        
        # 전체 데이터 정규화
        all_landmarks = np.array(all_landmarks)
        all_landmarks_scaled = self.scaler.fit_transform(all_landmarks)
        
        # 정규화된 데이터를 다시 시퀀스로 재구성
        for i in range(len(self.sequences)):
            start = i * self.sequence_length
            end = start + self.sequence_length
            self.sequences[i] = all_landmarks_scaled[start:end]
        
        if self.sequences:
            print(f"sequence shape: {self.sequences[0].shape}")
        else:
            print("sequences 생성 실패")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        if idx >= len(self.sequences):
            raise IndexError(f"Index {idx} out of range. Dataset length: {len(self.sequences)}")
        sequence = self.sequences[idx]
        return torch.FloatTensor(sequence), torch.LongTensor([self.labels[idx]]).squeeze()

# GRU 모델 정의
class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=3, dropout=0.5):
        super(FallDetectionGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# 데이터 로드 및 전처리
json_folder = 'D:\\human_fall\\re_landmark\\addition_json'
json_files = [os.path.join(json_folder, f) for f in os.listdir(json_folder) if f.endswith('.json')]
dataset = FallSequenceDataset(json_files)

# 데이터셋을 학습 및 검증 세트로 분할
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

# 데이터 로더 생성 전에 클래스 가중치 계산
class_weights = compute_class_weight('balanced', classes=np.unique(dataset.labels), y=dataset.labels)
class_weights = torch.FloatTensor(class_weights).to(device)

# 손실 함수에 가중치 적용
criterion = nn.CrossEntropyLoss(weight=class_weights)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

if len(dataset) > 0:
    sample_sequence, sample_label = dataset[0]
    input_size = sample_sequence.shape[1]
    print(f'input_size :', {input_size})
    model = FallDetectionGRU(input_size).to(device)
else:
    print("데이터 없음")
    exit()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
num_epochs = 500
best_loss = float('inf')
patience = 50
no_improve = 0

for epoch in range(num_epochs):
    model.train()
    total_loss_train = 0
    
    for sequences, labels in train_loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(sequences)
        loss_train = criterion(outputs, labels.view(-1))
        loss_train.backward()
        optimizer.step()
        
        total_loss_train += loss_train.item()
    
    avg_loss_train = total_loss_train / len(train_loader)

    # 검증 단계
    model.eval()
    total_loss_val = 0
    
    with torch.no_grad():
        for sequences_val, labels_val in val_loader:
            sequences_val, labels_val = sequences_val.to(device), labels_val.to(device)
            outputs_val = model(sequences_val)
            loss_val = criterion(outputs_val, labels_val.view(-1))
            total_loss_val += loss_val.item()
    
    avg_loss_val = total_loss_val / len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss_train:.4f}, Val Loss: {avg_loss_val:.4f}')
    scheduler.step(avg_loss_val)
    
    if avg_loss_val < best_loss:
        best_loss = avg_loss_val
        no_improve = 0
        torch.save(model.state_dict(), 'add_sensordata.pt')
    else:
        no_improve += 1
    
    if no_improve >= patience:
        print("Early stopping")
        break

def calculate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for sequences, labels in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)
    return f1, cm

# 학습 루프 내에서 성능 지표 계산
train_f1, train_cm = calculate_metrics(model, train_loader)
val_f1, val_cm = calculate_metrics(model, val_loader)
print(f'Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')
print(f'Train CM:\n{train_cm}\nVal CM:\n{val_cm}')

print("학습 완료")

LANDMARKS length: 11
시퀀스 생성 실패
Processing file: D:\human_fall\re_landmark\addition_json\00060_H_A_SY_C1.json
Processing file: D:\human_fall\re_landmark\addition_json\00060_H_A_SY_C2.json
Processing file: D:\human_fall\re_landmark\addition_json\00060_H_A_SY_C3.json
Processing file: D:\human_fall\re_landmark\addition_json\00060_H_A_SY_C4.json
Processing file: D:\human_fall\re_landmark\addition_json\00060_H_A_SY_C5.json
Processing file: D:\human_fall\re_landmark\addition_json\00060_H_A_SY_C6.json
Processing file: D:\human_fall\re_landmark\addition_json\00060_H_A_SY_C7.json
Processing file: D:\human_fall\re_landmark\addition_json\00060_H_A_SY_C8.json
Processing file: D:\human_fall\re_landmark\addition_json\00093_H_A_FY_C1.json
Processing file: D:\human_fall\re_landmark\addition_json\00093_H_A_FY_C2.json
Processing file: D:\human_fall\re_landmark\addition_json\00093_H_A_FY_C3.json
Processing file: D:\human_fall\re_landmark\addition_json\00093_H_A_FY_C4.json
Processing file: D:\human_fall\re

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/500], Train Loss: 0.6247, Val Loss: 0.5111
Epoch [2/500], Train Loss: 0.5188, Val Loss: 0.4659
Epoch [3/500], Train Loss: 0.4897, Val Loss: 0.4574
Epoch [4/500], Train Loss: 0.4759, Val Loss: 0.4505
Epoch [5/500], Train Loss: 0.4655, Val Loss: 0.4248
Epoch [6/500], Train Loss: 0.4531, Val Loss: 0.4170
Epoch [7/500], Train Loss: 0.4441, Val Loss: 0.4104
Epoch [8/500], Train Loss: 0.4347, Val Loss: 0.4122
Epoch [9/500], Train Loss: 0.4297, Val Loss: 0.4121
Epoch [10/500], Train Loss: 0.4253, Val Loss: 0.3973
Epoch [11/500], Train Loss: 0.4151, Val Loss: 0.3953
Epoch [12/500], Train Loss: 0.4120, Val Loss: 0.3825
Epoch [13/500], Train Loss: 0.4025, Val Loss: 0.3885
Epoch [14/500], Train Loss: 0.4001, Val Loss: 0.3823
Epoch [15/500], Train Loss: 0.3964, Val Loss: 0.3800
Epoch [16/500], Train Loss: 0.3921, Val Loss: 0.3758
Epoch [17/500], Train Loss: 0.3886, Val Loss: 0.3712
Epoch [18/500], Train Loss: 0.3826, Val Loss: 0.3686
Epoch [19/500], Train Loss: 0.3822, Val Loss: 0.3793
Ep

### sensordata와 yolo bbox 좌표를 동시에 training
* 클래스 예측은 랜드마크의 좌표값을 우선적으로, 차선으로 bbox 좌표로 분류하도록 수정

In [10]:
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# bbox의 비율 계산 함수
def bbox_ratio(bbox) : 
    w = bbox['x2'] - bbox['x1']
    h = bbox['y2'] - bbox['y1']
    return w / h if w != 0 else 0

# bbox의 비율에 따른 클래스 분류 
def bbox_ratio_class(ratio) :
    if ratio < 0.5 : 
        return 0 # Normal 클래스
    elif 0.5 < ratio < 0.7 : 
        return 1 # Danger 클래스
    else : 
        return 2 # Fall 클래스 
    
# 데이터셋 클래스 정의
class FallSequenceDataset(Dataset):
    def __init__(self, json_files, sequence_length=3):
        self.sequence_length = sequence_length
        self.sequences = []
        self.labels = []
        self.scaler = StandardScaler()
        print(f"LANDMARKS length: {len(LANDMARKS)}")
           
        all_landmarks = []
        
        for json_file in json_files:
            print(f'Processing file: {json_file}')
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            frames = list(data['pose_data'].values())
            
            for i in range(0, len(frames) - self.sequence_length + 1):
                sequence = frames[i:i+self.sequence_length]
                landmarks = []
                
                for frame in sequence:
                    frame_landmarks = []
                    for landmark in LANDMARKS:
                        frame_landmarks.extend([
                            frame[f'landmark_{landmark}']['x'],
                            frame[f'landmark_{landmark}']['y']
                        ])
                        
                    # YOLO bbox 좌표 추가
                    try:
                        bbox = frame['bbox']
                        frame_landmarks.extend([
                            frame['bbox']['x1'],
                            frame['bbox']['y1'],
                            frame['bbox']['x2'],
                            frame['bbox']['y2']
                        ])
                        
                        # bbox 비율에 따른 클래스 결정
                        b_ratio = bbox_ratio(bbox)
                        label = bbox_ratio_class(b_ratio)
                        
                    except KeyError:
                        print('bbox 정보 없음. 기본값으로 사용')
                        frame_landmarks.extend([0, 0, 1, 1])  # 기본값 사용
                        
                    landmarks.append(frame_landmarks)
                
                all_landmarks.extend(landmarks)
                
                # 마지막 프레임의 클래스를 레이블로 사용
                label = 0 if frame['class'] == 'Normal' else (1 if frame['class'] == 'Danger' else 2)
                
                self.sequences.append(landmarks)
                self.labels.append(label)
        
        # 전체 데이터 정규화
        all_landmarks = np.array(all_landmarks)
        all_landmarks_scaled = self.scaler.fit_transform(all_landmarks)
        
        # 정규화된 데이터를 다시 시퀀스로 재구성
        for i in range(len(self.sequences)):
            start = i * self.sequence_length
            end = start + self.sequence_length
            self.sequences[i] = all_landmarks_scaled[start:end]
        
        if self.sequences:
            print(f"sequence shape: {self.sequences[0].shape}")
            print(f"Features per frame: {self.sequences[0].shape[1]}")
        else:
            print("sequences 생성 실패")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        if idx >= len(self.sequences):
            raise IndexError(f"Index {idx} out of range. Dataset length: {len(self.sequences)}")
        sequence = self.sequences[idx]
        return torch.FloatTensor(sequence), torch.LongTensor([self.labels[idx]]).squeeze()

# GRU 모델 정의
class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=3, dropout=0.5):
        super(FallDetectionGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# 데이터 로드 및 전처리
json_folder = 'D:\\human_fall\\re_landmark\\addition_yolobbox_json'
json_files = [os.path.join(json_folder, f) for f in os.listdir(json_folder) if f.endswith('.json')]
dataset = FallSequenceDataset(json_files)

# 데이터셋을 학습 및 검증 세트로 분할
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

# 데이터 로더 생성 전에 클래스 가중치 계산
class_weights = compute_class_weight('balanced', classes=np.unique(dataset.labels), y=dataset.labels)
class_weights = torch.FloatTensor(class_weights).to(device)

# 손실 함수에 가중치 적용
criterion = nn.CrossEntropyLoss(weight=class_weights)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

if len(dataset) > 0:
    sample_sequence, sample_label = dataset[0]
    input_size = sample_sequence.shape[1]
    print(f'input_size : {input_size}')
    model = FallDetectionGRU(input_size).to(device)
else:
    print("데이터 없음")
    exit()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
num_epochs = 500
best_loss = float('inf')
patience = 50
no_improve = 0

for epoch in range(num_epochs):
    model.train()
    total_loss_train = 0
    
    for sequences, labels in train_loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(sequences)
        loss_train = criterion(outputs, labels.view(-1))
        loss_train.backward()
        optimizer.step()
        
        total_loss_train += loss_train.item()
    
    avg_loss_train = total_loss_train / len(train_loader)

    # 검증 단계
    model.eval()
    total_loss_val = 0
    
    with torch.no_grad():
        for sequences_val, labels_val in val_loader:
            sequences_val, labels_val = sequences_val.to(device), labels_val.to(device)
            outputs_val = model(sequences_val)
            loss_val = criterion(outputs_val, labels_val.view(-1))
            total_loss_val += loss_val.item()
    
    avg_loss_val = total_loss_val / len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss_train:.4f}, Val Loss: {avg_loss_val:.4f}')
    scheduler.step(avg_loss_val)
    
    if avg_loss_val < best_loss:
        best_loss = avg_loss_val
        no_improve = 0
        torch.save(model.state_dict(), 'add_sensordata_bbox.pt')
    else:
        no_improve += 1
    
    if no_improve >= patience:
        print("Early stopping")
        break

def calculate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for sequences, labels in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)
    return f1, cm

# 학습 루프 내에서 성능 지표 계산
train_f1, train_cm = calculate_metrics(model, train_loader)
val_f1, val_cm = calculate_metrics(model, val_loader)
print(f'Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')
print(f'Train CM:\n{train_cm}\nVal CM:\n{val_cm}')

print("학습 완료")

LANDMARKS length: 11
Processing file: D:\human_fall\re_landmark\addition_yolobbox_json\00060_H_A_SY_C1.json
Processing file: D:\human_fall\re_landmark\addition_yolobbox_json\00060_H_A_SY_C2.json
Processing file: D:\human_fall\re_landmark\addition_yolobbox_json\00060_H_A_SY_C3.json
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
Processing file: D:\human_fall\re_landmark\addition_yolobbox_json\00060_H_A_SY_C4.json
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
bbox 정보 없음. 기본값으로 사용
b

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/500], Train Loss: 0.5996, Val Loss: 0.4709
Epoch [2/500], Train Loss: 0.4966, Val Loss: 0.4473
Epoch [3/500], Train Loss: 0.4559, Val Loss: 0.3990
Epoch [4/500], Train Loss: 0.4310, Val Loss: 0.3873
Epoch [5/500], Train Loss: 0.4142, Val Loss: 0.3707
Epoch [6/500], Train Loss: 0.4031, Val Loss: 0.3650
Epoch [7/500], Train Loss: 0.3913, Val Loss: 0.3582
Epoch [8/500], Train Loss: 0.3819, Val Loss: 0.3400
Epoch [9/500], Train Loss: 0.3723, Val Loss: 0.3343
Epoch [10/500], Train Loss: 0.3650, Val Loss: 0.3203
Epoch [11/500], Train Loss: 0.3564, Val Loss: 0.3342
Epoch [12/500], Train Loss: 0.3496, Val Loss: 0.3294
Epoch [13/500], Train Loss: 0.3419, Val Loss: 0.3298
Epoch [14/500], Train Loss: 0.3362, Val Loss: 0.3135
Epoch [15/500], Train Loss: 0.3342, Val Loss: 0.3096
Epoch [16/500], Train Loss: 0.3276, Val Loss: 0.3067
Epoch [17/500], Train Loss: 0.3205, Val Loss: 0.3042
Epoch [18/500], Train Loss: 0.3139, Val Loss: 0.3170
Epoch [19/500], Train Loss: 0.3097, Val Loss: 0.2959
Ep